## 5400 Group Project - Group 11

### This is the code where we used MongoDB and Spark for ETL and flask for front-end development

### Back-End: ETL in Spark and Mongodb

    Process and store the downloaded csv data used for this project in spark and mongodb

Data Preprocessing in Spark

In [ ]:
%pip install pyspark

In [ ]:
# from pyspark.sql import SparkSession

# spark = SparkSession.builder.getOrCreate()
# sc = spark.sparkContext
# print("Using Apache Spark Version", spark.version)

# import os, sys

# os.environ['PYSPARK_PYTHON'] = sys.executable
# os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [1]:
# from pymongo import MongoClient
# client = MongoClient('localhost',27017) 
# db = client.apan5400 
# collection_name = 'project' # The collection is called 'project'
# collection = db.collection_name

In [1]:
from pyspark.sql import SparkSession

# Initialize Spark Session with MongoDB Connector
spark = SparkSession.builder \
    .appName("DataFrame to MongoDB") \
    .config("spark.driver.memory","6g") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/apan5400.project") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/apan5400.project") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .getOrCreate()

:: loading settings :: url = jar:file:/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/apple/.ivy2/cache
The jars for the packages stored in: /Users/apple/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a0f7be36-f48e-4249-bd88-f952f69c55ba;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 525ms :: artifacts dl 32ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts

In [2]:
# Deinfe a function to manipulate columns: delete useless columns and rename columns
from pyspark.sql import DataFrame # import functions needed

def cleaned_sp(data: DataFrame) -> DataFrame:
    """
    Process the input DataFrame by renaming a column and dropping several others,
    preparing it for insertion into MongoDB.

    Args:
    data (DataFrame): The input Spark DataFrame.

    Returns:
    DataFrame: The processed Spark DataFrame ready for MongoDB insertion.
    """
    # rename the column 'Violation' with 'Violation Description' for user purpose
    renamed = data.withColumnRenamed('Violation', 'Description')
    # list of columns to drop
    dropped = ['Judgment Entry Date', 'Precinct', 'County', 'Issuing Agency', 'Violation Status']
    # drop the columns listed above
    cleaned = renamed.drop(*dropped)
    # return the converted data
    return cleaned

In [3]:
data = spark.read.option("header", "true") \
                   .option("delimiter", ",") \
                   .option("inferSchema", "true") \
                   .csv("/Users/apple/Desktop/54 project/subset_1_5gb.csv")

# Process and convert data
cleaned_data = cleaned_sp(data)

In [11]:
# Inspect the cleaned data
print(cleaned_data.count())
print(cleaned_data.columns)

5203278
['Plate', 'State', 'License Type', 'Summons Number', 'Issue Date', 'Violation Time', 'Description', 'Fine Amount', 'Penalty Amount', 'Interest Amount', 'Reduction Amount', 'Payment Amount', 'Amount Due', 'Summons Image']


Data Storage in Mongodb

In [ ]:
%pip install setuptools

In [ ]:
from pyspark.sql import DataFrame
import pandas as pd

def convert_to_mongo_docs(spark_df: DataFrame) -> list:
    try:
     # Convert Spark DataFrame to Pandas DataFrame
     cleaned_data_pd = cleaned_data.toPandas()
     print("Converted Spark DataFrame to Pandas DataFrame.")
    except Exception as e:
        print(f"Failed to convert to Pandas DataFrame: {e}")

    # Convert Pandas DataFrame to list of dictionaries
    data_dicts = cleaned_data_pd.to_dict('records')
    print("Converted Pandas DataFrame to list of dictionaries.")

    return data_dicts

In [ ]:
convert_to_mongo_docs(cleaned_data)

In [ ]:
# # convert cleaned data to dicts for MongoDB insertion
# import pandas as pd

# try:
#      # Convert Spark DataFrame to Pandas DataFrame
#      cleaned_data_pd = cleaned_data.toPandas()
#      print("Converted Spark DataFrame to Pandas DataFrame.")
# except Exception as e:
#      print(f"Failed to convert to Pandas DataFrame: {e}")

#  # Convert Pandas DataFrame to list of dictionaries
# data_dicts = cleaned_data_pd.to_dict('records')
# print("Converted Pandas DataFrame to list of dictionaries.")

In [ ]:
# method 2: use Spark MongoDB connector
# Write the processed data to MongoDB
# cleaned_data.write.format("mongodb").mode("overwrite").save()

# Stop the Spark session
# spark.stop()

    Enable the update function in mongodb so that database administrators can update the datasource driectly fetched from API url according on demand. 

In [5]:
# Define a function to fetch data directly from the API url on NYC Open Data website
import requests # import functions needed
from pyspark.sql import SparkSession
from pyspark.sql import Row

def fetch_data_api():
    # Data fetched from this API url below will be in csv format
    api_url = 'https://data.cityofnewyork.us/resource/nc67-uf89.csv'

    response = requests.get(api_url) # Send GET request to the API
    response.raise_for_status()  # Exception for HTTP errors

    # Save the CSV data to a local file
    with open('data.csv', 'w') as file:
        file.write(response.text)

    # Initialize Spark Session
    spark = SparkSession.builder.appName('Load CSV Data').getOrCreate()

    # Load the CSV file into a DataFrame
    df = spark.read.csv('data.csv', header=True, inferSchema=True)

    return df

In [ ]:
# Connect to mongodb
from pymongo import MongoClient
client = MongoClient('localhost',27017) 
db = client.apan5400 
collection_name = 'project' # The collection is called 'project'

# Check if the collection called 'project' exists, drop it if it does, o/w create a new one
def recreate_collection():
    # if the collection already exists
    if collection_name in db.list_collection_names():
        print(f"Collection '{collection_name}' already exists. Dropping it now.")
        # drop the existing collection
        db[collection_name].drop()
    # if the collection does not exist
    print(f"Creating new collection '{collection_name}'.")
    # create a new collection for further data insertion
    collection = db[collection_name]
    return collection
    
# Function to insert data into the collection 'project'
def insert_data(collection, data_dict):
    try:
        # insert documents
        result = collection.insert_many(data_dict)
        print(f"Inserting documents into collection '{collection_name}'")
    except Exception as e:
        print(f"Error inserting data: {e}")

# Function to run the ETL process: fetch data from API and overwrite the data in mongodb
def update_mongodb():
    collection = recreate_collection()
    data = fetch_data_api()
    cleaned_data = cleaned_sp(data)
    dicts=convert_to_mongo_docs(cleaned_data)
    insert_data(collection, dicts)
    print('Update Done')


### Front-End: Flask 

In [ ]:
pip install Flask

In [3]:
import requests, json
from flask import Flask, render_template, request, jsonify, redirect, url_for, flash
from pymongo import MongoClient

app = Flask("")
#don't delete this row
app.secret_key = 'secret_key123'
#response = requests.get('https://data.cityofnewyork.us/resource/nc67-uf89.json')

client = MongoClient('localhost',27017) 
db = client.apan5400 
collection = db.project

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        plate_number = request.form.get('plate')
        state = request.form.get('state')
        if not plate_number:
            flash('Please enter a plate number.', 'error')
            return redirect(url_for('index'))
        return redirect(url_for('view_violations', plate=plate_number.strip()))
    return render_template('index.html')

@app.route('/view_violations/<plate>', methods=['GET'])
def view_violations(plate):
    violations = list(collection.find({"Plate": plate}, {'_id': 1, 'Issue Date': 1, 'Description': 1, 'Amount Due': 1}))
    if not violations:
        flash('No records found for the provided plate number.', 'error')
        return redirect(url_for('index'))
    return render_template('violations_new.html', plate=plate, violations=violations)


@app.route('/view_by_summons', methods=['GET'])
def view_by_summons():
    summons_number = request.args.get('summons')
    if summons_number:
        # Assuming 'Summons Number' is stored as an integer in your MongoDB.
        # If it's stored as a string, you should remove the int() cast.
        violation = collection.find_one({"Summons Number": int(summons_number)}, {'_id': 0})

        if not violation:
            flash('No records found for the provided summons number.', 'error')
            return redirect(url_for('index'))
        
        plate = violation.get('Plate')
        issue_date = violation.get('Issue Date')

        # Redirect to violation_detail with the found plate and issue date
        return redirect(url_for('violation_detail', plate=plate, issue_date=issue_date.replace('/', '-')))
    else:
        flash('Please enter a summons number.', 'error')
        return redirect(url_for('index'))



@app.route('/violation_detail/<plate>/<issue_date>', methods=['GET'])
def violation_detail(plate, issue_date):
    from datetime import datetime
     # Convert issue_date from '07-11-2019' to '07/11/2019'
    formatted_issue_date = datetime.strptime(issue_date, '%m-%d-%Y').strftime('%m/%d/%Y')

    # print(f"Looking up violation for plate {plate} on date {formatted_issue_date}")
    violation = collection.find_one({"Plate": plate, "Issue Date": formatted_issue_date}, {'_id': 0})
    # print(f"Violation found: {violation}")

    if not violation:
        flash('No details found for this violation.', 'error')
        return redirect(url_for('view_violations', plate=plate))
    
    import ast
        # If the summons image is stored as a string that looks like a dictionary
    if violation and isinstance(violation.get('Summons Image'), str):
        # Safely evaluate the string to a dictionary
        try:
            violation['Summons Image'] = ast.literal_eval(violation['Summons Image'])
        except ValueError:
            # If there is an error during evaluation, set it to None or {}
            violation['Summons Image'] = None

    # Render your violation_detail template here
    return render_template('violation_detail.html', violation=violation, plate=plate, issue_date=formatted_issue_date)


@app.route('/pay', methods=['GET'])
def show_payment_form():
    plate = request.args.get('plate', '')
    issue_date = request.args.get('issue_date', '')
    amount_due = request.args.get('amount_due', '0')
    
    print(f"Received payment for plate {plate} on {issue_date} for amount ${amount_due}")
    
    return render_template('payment_form.html', plate=plate, issue_date=issue_date, amount_due=amount_due)



from flask import flash, redirect, url_for

@app.route('/submit_payment', methods=['POST'])
def submit_payment():
    
    from decimal import Decimal
    # Extract the payment information from the form submission
    plate = request.form.get('plate')
    issue_date = request.form.get('issue_date')
    amount_paid = Decimal(request.form.get('amount')).quantize(Decimal('0.01'))  # Round to 2 decimal places
        # Simulate payment processing
    print(f"Processing payment for plate {plate} on {issue_date} for amount ${amount_paid}")

    # Fetch the current amount due from MongoDB
    current_data = collection.find_one({"Plate": plate, "Issue Date": issue_date}, {'_id': 0, 'Amount Due': 1})

    if current_data is None:
        flash('No violation record found for this plate and date.', 'error')
        return redirect(url_for('show_payment_form', plate=plate, issue_date=issue_date))

    # Calculate the new amount due
    current_amount_due = Decimal(str(current_data['Amount Due'])).quantize(Decimal('0.01'))  # Convert and round
    new_amount_due = (current_amount_due - amount_paid).quantize(Decimal('0.01'))  # Calculate and round
    new_amount_due = float(new_amount_due)

    print(f"Old Amount Due: ${current_amount_due}, Amount Paid: ${amount_paid}, New Amount Due: ${new_amount_due}")

    # Update the database with the new amount due
    collection.update_one(
        {"Plate": plate, "Issue Date": issue_date},
        {"$set": {"Amount Due": new_amount_due}}
    )
    return render_template('submit_payment.html', plate=plate, issue_date=issue_date, amount_paid=amount_paid)


app.run(host='localhost', port=5002) 


 * Serving Flask app ''
 * Debug mode: off


 * Running on http://localhost:5002
[2024-04-21 22:47:45,428] INFO in _internal: WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://localhost:5002
Press CTRL+C to quit
[2024-04-21 22:47:45,430] INFO in _internal: Press CTRL+C to quit
[2024-04-21 22:47:52,829] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/flask/app.py", line 1463, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/flask/app.py", line 872, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/flask/app.py", line 870, in full_dispatch_request
    rv = self